# 数据透视和数据重塑
## 数据透视

`pivot_table` 是用于数据透视的重要函数之一。它根据一个或多个列的值对数据进行重新排列和汇总，以便更好地理解数据的结构和关系，等同于 Excel 数据透视表的功能，能够处理更大量数据且功能更强大（可自定义函数进行聚合）。

![数据透视](../drawio/ch-pandas/pivot.drawio)
:width:`800px`
:label:`pivot-img`

In [ ]:
import pandas as pd
import numpy as np
sales_df = pd.read_excel("/Users/xu/Downloads/sales-funnel.xlsx")
sales_df

: 

pivot_table函数的完整形式为`pd.pivot_table(data, values=None, index=None, columns=None, aggfunc='mean', fill_value=None, margins=False, dropna=True, margins_name='All', observed=False)`。参数很多，只有`data`是唯一必需的参数，其他均有默认值，常需要设置的参数有 `values`,`index`,`columns`,`aggfunc`等。

### index和columns参数

index: 用于分组数据，生成透视表的行索引，可以为多个属性。

columns: 用于分组数据，生成透视表的列索引。

例1: 按照 Name 分组，即添加 Name 的行索引。

In [ ]:
pd.pivot_table(sales_df,index=['Name'])

例2: 按 Manager 和 Rep 分组，类似于 `groupby Manager, Rep`。

In [ ]:
pd.pivot_table(sales_df, index=['Manager','Rep'])

例3: 添加透视表的列索引 Product。

In [ ]:
pd.pivot_table(sales_df, index=['Manager','Rep'], columns=["Product"])

### values参数
values参数指明要聚合的列或列的列表，通常是你想要在透视表中分析的数值数据。

例：只显示 Price 列对应的平均值。

In [ ]:
pd.pivot_table(sales_df,index=['Manager','Rep'],values='Price')

### aggfunc参数
用于汇总数据的聚合函数，可以是字符串（如 'sum'、'mean'、'count' 等）或自定义聚合函数。

例：对 Price 显示 mean 和 len(计数) 两列结果。

In [ ]:
pd.pivot_table(sales_df,index=["Manager","Rep"],values=["Price"],aggfunc=[np.mean,len])

### fill_value 和 margins参数
fill_value 用于填充缺失值的值; margins 为布尔值，如果为 True, 则在透视表中包含行和列的总计。

In [ ]:
pd.pivot_table(sales_df,index=["Manager","Rep","Product"],
               values=["Price","Quantity"],
               aggfunc=[np.sum,np.mean],fill_value=0,margins=True)

### 维度交叉

将以上参数结合起来，对 Quantity 和 Price 应用不同的聚合函数。

table = pd.pivot_table(sales_df,index=["Manager","Status"],columns=["Product"],values=["Quantity","Price"],
               aggfunc={"Quantity":len,"Price":[np.mean]},fill_value=0)
table

### 数据筛选
pandas中，建立数据透视表（pivot table）后，也可以进一步筛选数据以满足特定的条件，类似于 DataFrame 的数据切片方法中的条件索引。这里主要介绍 `.query`方法，允许通过传递查询字符串来筛选数据。

例1: 筛选出 Manager 为 Debra Henley 的结果。

In [ ]:
table.query('Manager == ["Debra Henley"]')

例2: 筛选出 Status 为 pending 或 won 的结果。

In [ ]:
table.query('Status == ["pending","won"]')

## 数据重塑
pandas.melt()是用于将宽格式（wide format）的数据表格转换为长格式（long format）。这个函数通常用于数据重塑（data reshaping）操作，以便更容易进行数据分析和可视化。

通过这种方式，可以将宽格式数据表格中的多列数据整合到一个列中，可以理解为透视表pivot_table()的反操作。

![数据重塑](../drawio/ch-pandas/melt.drawio)
:width:`800px`
:label:`melt-img`

In [ ]:
import pandas as pd
#create student_df
d1 = {'Student_id':pd.Series([1,2,3]), 'Name':pd.Series(['Amy','Bob','John']), 
'Math':pd.Series([90,78,88]),'English':pd.Series([85,92,79]),
'History':pd.Series([88,76,90])}
student_df = pd.DataFrame(d1)
print("student_df:")
student_df

In [ ]:
melted_df = pd.melt(student_df, id_vars=['Student_id', 'Name'], value_vars=['Math', 'English', 'History'], 
var_name='Subject', value_name='Score')
print("melted_df")
melted_df

数据重塑时，通常需要使用`value_vars`来指定需要“融化”的列，使它们被整合为一列。比如本例中，将`Math`,`English`,`History`作为`Subject`被整合为一列。

我们回到 `melt` 函数的参数。完整的参数形式为:`pandas.melt(frame, id_vars=None, value_vars=None, var_name=None, value_name='value', col_level=None)`。

`frame`指的是要进行重塑操作的 DataFrame 。

`id_vars`表示需要保留的列，它们将成为长格式中的标识变量（identifier variable），不被"融化"。

`value_vars`是需要"融化"的列，它们将被整合成一列，并用新的列名表示。

`var_name`用于存储"融化"后的列名的新列的名称。

`value_name`用于存储"融化"后的值的新列的名称。

如果输入数据是多级索引（MultiIndex），则可以用`col_level`指定在哪个级别上应用"融化"操作。
